In [1]:
import pymongo
import pandas as pd
import numpy as np
import re
import time
import datetime
import nltk
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bson.objectid import ObjectId

In [2]:
client = pymongo.MongoClient('localhost',27017)
db = client.db
threads = db.threads
companies = db.companylist
matches = db.matches
chunks = db.chunks
nltk.download('vader_lexicon')

df = pd.DataFrame.from_records(threads.find({'Label':{'$ne': 0}}))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
sir = SentimentIntensityAnalyzer()
sent = df.loc[77,'Title'] + ' ' + df.loc[77,'Body']
sir.polarity_scores(sent)['compound']
count = 0

for i in range(len(df)):
    sent = df.loc[i,'Title'] + ' ' + df.loc[i,'Body']
    score = sir.polarity_scores(sent)['compound']
    if (score < 0 and df.loc[i, 'Label'] == 'bullish') or (score > 0 and df.loc[i,'Label'] == 'bearish'):
        print(str(score) + ':' + df.loc[i, 'Label'])
        count = count + 1

-0.9077:bullish
-0.7453:bullish
-0.4522:bullish
-0.9222:bullish
0.9274:bearish
0.8258:bearish
0.1027:bearish
-0.6037:bullish
-0.9495:bullish
0.8022:bearish
-0.5267:bullish
-0.5994:bullish
0.3716:bearish
0.8689:bearish
-0.743:bullish
0.4791:bearish
-0.608:bullish
0.2263:bearish
-0.8371:bullish
-0.8185:bullish
0.6326:bearish
0.9801:bearish
-0.0874:bullish
-0.953:bullish
-0.25:bullish
-0.5574:bullish
0.5978:bearish
0.9976:bearish
-0.8658:bullish
-0.4246:bullish
-0.9578:bullish
-0.3182:bullish
-0.1027:bullish
-0.5829:bullish
-0.2313:bullish
-0.8578:bullish
0.7506:bearish
0.7063:bearish
-0.5789:bullish
-0.4939:bullish
0.8266:bearish
-0.3533:bullish
-0.4767:bullish
-0.3249:bullish
-0.1927:bullish
0.4404:bearish
0.323:bearish
-0.7814:bullish
0.296:bearish
0.9917:bearish
-0.5106:bullish
-0.599:bullish
0.8943:bearish
0.8957:bearish
-0.6369:bullish
0.9042:bearish
-0.5106:bullish
0.8236:bearish
0.9208:bearish
-0.1027:bullish
0.6739:bearish
0.9169:bearish
-0.3972:bullish
-0.1655:bullish
0.9617:bea

In [4]:
import flair
from flair.models import TextClassifier
from flair.data import Sentence

def cleanText(text):
    return re.sub("http[s]?://\S+", ' ', text) #Remove links

In [5]:
classifier = TextClassifier.load('en-sentiment')
sentence = Sentence(sent)
classifier.predict(sentence)

2020-10-30 00:20:56,669 loading file /home/user/.flair/models/sentiment-en-mix-distillbert_3.1.pt


In [6]:
labelDf = pd.DataFrame(columns = ['Label', 'Text'])
labelDf['Text'] = df['Title'] + ' ' + df['Body']
labelDf['Label'] = df['Label']
labelDf['Text'] = labelDf['Text'].apply(cleanText)

In [7]:
labelDf['Label'] = '__label__' + labelDf['Label'].astype(str)

In [8]:
labelDf = labelDf.sample(frac = 1)

In [10]:
labelDf.iloc[0: int(len(labelDf)*0.8)].to_csv('data/train.csv', sep = '\t', index = False, header = False)
labelDf.iloc[int(len(labelDf)*0.8): int(len(labelDf)*0.9)].to_csv('data/test.csv', sep = '\t', index = False, header = False)
labelDf.iloc[int(len(labelDf)*0.9): ].to_csv('data/dev.csv', sep = '\t', index = False, header = False)

In [7]:
'''from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
corpus = ClassificationCorpus(Path('data/'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
word_embeddings = [WordEmbeddings('glove')]
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train('model/', max_epochs=10)'''

2020-10-30 11:07:11,645 Reading data from data
2020-10-30 11:07:11,647 Train: data/train.csv
2020-10-30 11:07:11,648 Dev: data/dev.csv
2020-10-30 11:07:11,655 Test: data/test.csv
2020-10-30 11:07:13,031 Computing label dictionary. Progress:
100%|██████████| 971/971 [00:03<00:00, 272.96it/s]2020-10-30 11:07:18,260 [b'bearish', b'bullish', b'neutral']
2020-10-30 11:07:18,263 ----------------------------------------------------------------------------------------------------
2020-10-30 11:07:18,271 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
    )
    (word_reprojection_map): Linear(in_features=100, out_features=256, bias=True)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=3, bias=True)
  (loss_function): CrossEntropyLoss()
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-10-30 11:07:

In [2]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('model/best-model.pt')
sentence = Sentence('stonks going up')
classifier.predict(sentence)
print(sentence.labels)

2020-10-30 13:33:38,378 loading file model/best-model.pt
[bullish (0.5738)]
